# Transfer learning
Firstly training the model to classify the numerical digits ranging from (0 - 9)
then would be using the same pretrained model using the concept of Transfer Learning to classify even and odd digits.

In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

2023-07-22 16:39:06.727314: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-22 16:39:06.819443: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-22 16:39:06.820888: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-22 16:39:08.635490: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [5]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [6]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266610 (1.02 MB)
Trainable params: 266610 

In [8]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=1)

2023-07-22 16:41:32.113136: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 172480000 exceeds 10% of free system memory.


Epoch 1/10
1719/1719 [==============================] - 11s 6ms/step - loss: 1.5228 - accuracy: 0.6067 - val_loss: 0.9294 - val_accuracy: 0.7962
Epoch 2/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.7401 - accuracy: 0.8244 - val_loss: 0.5874 - val_accuracy: 0.8562
Epoch 3/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.5433 - accuracy: 0.8585 - val_loss: 0.4723 - val_accuracy: 0.8782
Epoch 4/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4618 - accuracy: 0.8751 - val_loss: 0.4134 - val_accuracy: 0.8892
Epoch 5/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4162 - accuracy: 0.8853 - val_loss: 0.3777 - val_accuracy: 0.8988
Epoch 6/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.3865 - accuracy: 0.8921 - val_loss: 0.3534 - val_accuracy: 0.9034
Epoch 7/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3651 - accuracy: 0.8977 - val_loss: 0.3353 - val_ac

In [9]:
model.save("pretrained_mnist_model.h5")

/home/usman/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Loading the model trained above on mnist dataset to identify the hand written numerical digits.

In [11]:
pretrained_mnist_model = tf.keras.models.load_model("pretrained_mnist_model.h5")

In [12]:
pretrained_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266610 (1.02 MB)
Trainable params: 266610 

In [13]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten: True
dense: True
leaky_re_lu: True
dense_1: True
leaky_re_lu_1: True
dense_2: True


### Freezing all the layers of pretrained model except the last one to fine tune

In [14]:
for layer in pretrained_mnist_model.layers[:-1]:
  layer.trainable = False
  print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False


In [15]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False
dense_2: True


In [16]:
lower_pretrained_layers = pretrained_mnist_model.layers[:-1]

new_model = tf.keras.models.Sequential(lower_pretrained_layers)
new_model.add(
    tf.keras.layers.Dense(2, activation="softmax")
)

In [17]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 202       
                                                                 
Total params: 265802 (1.01 MB)
Trainable params: 202 (

In [18]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [21]:
def update_even_odd_labels(labels):
  for idx, label in enumerate(labels):
    labels[idx] = np.where(label % 2 == 0, 1, 0)
  return labels

In [22]:
# basic example to understand enumerate
alpha = ["a", "b", "c"]

for idx, val in enumerate(alpha):
  print(idx, val)

0 a
1 b
2 c


In [23]:
y_train_bin, y_test_bin, y_valid_bin = update_even_odd_labels([y_train, y_test, y_valid])

In [24]:
np.unique(y_valid_bin)

array([0, 1])

In [25]:
new_model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [26]:
history = new_model.fit(X_train, y_train_bin, epochs=10,
                    validation_data=(X_valid, y_valid_bin), verbose=1)

Epoch 1/10


2023-07-22 16:52:40.993442: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 172480000 exceeds 10% of free system memory.


1719/1719 [==============================] - 8s 4ms/step - loss: 0.4420 - accuracy: 0.7939 - val_loss: 0.3333 - val_accuracy: 0.8604
Epoch 2/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.3421 - accuracy: 0.8547 - val_loss: 0.3045 - val_accuracy: 0.8774
Epoch 3/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.3222 - accuracy: 0.8643 - val_loss: 0.2905 - val_accuracy: 0.8824
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3111 - accuracy: 0.8707 - val_loss: 0.2820 - val_accuracy: 0.8856
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3034 - accuracy: 0.8742 - val_loss: 0.2757 - val_accuracy: 0.8882
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2975 - accuracy: 0.8772 - val_loss: 0.2706 - val_accuracy: 0.8912
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2927 - accuracy: 0.8797 - val_loss: 0.2665 - val_accuracy: 0.8950
Epo

In [27]:
new_model.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 4ms/step - loss: 0.2741 - accuracy: 0.8923


[0.2740752696990967, 0.892300009727478]